In [1]:
import numpy as np
import pandas as pd
import sys, glob
import urllib3
import h5py
import cdsapi, zipfile, os, time
import warnings
import shutil
import xarray
from datetime import date
warnings.filterwarnings('ignore')

sys.path.append(os.getcwd()+'/../cds-backend/code/')
import cds_eua3 as eua

In [2]:
def request(rqdict):
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        'insitu-comprehensive-upper-air-observation-network',rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
#     z = zipfile.ZipFile('download.zip')
#     z.extractall(path='./test_requests/')
#     z.close()
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./test1/')
    z.close()
    
    files = glob.glob('./test1/*.nc')
    
    with eua.CDMDataset(files[0]) as f:
        data = f.to_dataframe()
    
    os.remove('download.zip')
    for i in files:
        os.remove(i)
    
    return data

In [3]:
def csv_request(rqdict):
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        'insitu-comprehensive-upper-air-observation-network',rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
#     z = zipfile.ZipFile('download.zip')
#     z.extractall(path='./test_requests/')
#     z.close()
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./test1/')
    z.close()
    
    files = glob.glob('./test1/*.csv*')
    
    data = pd.read_csv(files[0])
    
    os.remove('download.zip')
    for i in files:
        os.remove(i)
    
    return data

In [4]:
def cdmrequest(rqdict):
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        'insitu-comprehensive-upper-air-observation-network',rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
#     z = zipfile.ZipFile('download.zip')
#     z.extractall(path='./test_requests/')
#     z.close()
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./test1/')
    z.close()
    
    files = glob.glob('./test1/*.nc')
    
    data = eua.CDMDataset(files[0])
    
    os.remove('download.zip')
    for i in files:
        os.remove(i)
    
    return data

In [5]:
#
# returns the whole dataframe instead of just the first bit whenever there are more files retrieved from the cds
# only relevant, if you would like to have a look at the data
#
def multi_request(rqdict):
    t0 = time.time()

    c = cdsapi.Client()
    r = c.retrieve(
        'insitu-comprehensive-upper-air-observation-network',rqdict)
    if True:
        r.download(target='download.zip')
        assert os.stat('download.zip').st_size == r.content_length, "Downloaded file is incomplete"
#     z = zipfile.ZipFile('download.zip')
#     z.extractall(path='./test_requests/')
#     z.close()
    z = zipfile.ZipFile('download.zip')
    z.extractall(path='./test2/')
    z.close()
    
    datalist = []
    files = glob.glob('./test2/*.nc')
    with eua.CDMDataset(files[0]) as f:
        data = f.to_dataframe()
    if len(files) > 0:
        for i in files[1:]:
            with eua.CDMDataset(i) as f:
                add = f.to_dataframe()
            data = data.append(add, ignore_index=True)
#     if len(files) > 0:
#         for i in range(len(files)):
#             ds = xarray.open_dataset(files[i])            
#             to_be_removed = ['trajectory_index', 'trajectory']
#             for ivar in list(ds.variables):
#                 if 'string' in ivar:
#                     to_be_removed.append(ivar)

#                 if 'trajectory' in ds[ivar].dims and ivar not in list(ds.coords):
#                     report_id = ds[ivar].astype(object).sum(axis=1).astype(str)
#                     ds = ds.drop_vars(ivar)
#                     ds[ivar] = ('obs', report_id.values[ds.trajectory_index.values])  # todo obs ???

#                 if ds[ivar].ndim > 1:
#                     tmp= ds[ivar].astype(object).sum(axis=1).astype(str)
#                     ds = ds.drop_vars(ivar)
#                     idim = tmp.dims[0]
#                     ds[ivar] = (idim, tmp)

#             ds = ds.drop_vars(to_be_removed)
#             data = ds.to_dataframe()
#             datalist.append(data)
#         data = pd.concat(datalist, axis=0)
#     else:
#         files = glob.glob('./test2/*.csv')
#         data = pd.read_csv(files[0])
    
    os.remove('download.zip')
    try:
       shutil.rmtree('./test2/')
    except:
       print('Error while deleting directory')
    
    return data

In [6]:
def test_statid():
    log = []
    log.append('\n\n---STATID---')
    cachworkaround = np.random.random_sample()
    for s in [None, 'all', 11035, '11035', '0-20001-0-11035', '0-20100-0-00701','11*', '1103*', '0-20100*', ['11035', 10393], ['0-20001-0-11035', '0-20001-0-10393']]:
        print('statid - testing: ', s)
        rd={
            'variable':'air_temperature',
            'statid':s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try:
            print(rd)
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['110350', '1103', '0-20000-0-1103', '0-20000-0-110350']:
        print('statid - testing: ', s)
        rd={
            'variable':'air_temperature',
            'statid':s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        }
        try: 
            print(rd)
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('statid - testing: finished')
    return log

In [7]:
def test_variable():
    log = []
    log.append('\n\n---VARIABLE---')
    cachworkaround = np.random.random_sample()
    for s in ['air_temperature','dew_point_temperature', 'eastward_wind_speed', 'northward_wind_speed', 'relative_humidity', 'specific_humidity', 'wind_from_direction', 'wind_speed']:
        print('variable - testing: ', s)
        rd={
            'variable': s,
            'statid': '11035',
            'date': '20000101-20000131',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['temperature', 'dew_point', 'wind', 'humidity', 'sonde_type']:
        print('variable - testing: ', s)
        rd={
            'variable': s,
            'statid': '11035',
            'date': '20000101-20000131',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('variable - testing: finished')
    return log

In [8]:
def test_optional():
    log = []
    log.append('\n\n---OPTIONAL---')
    cachworkaround = np.random.random_sample()
    for s in ['sonde_type', 'bias_estimate', 'obs_minus_an', 'obs_minus_bg']:
        print('optional - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': ['11035'], # '0-20100-0-00701'],
            'date': '19781001-19781031',
            'optional': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ ' - ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['sonde', 'air_temperature', 'dew_point_temperature']:
        print('optional - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': ['11035'],# '0-20100-0-00701'],
            'date': '19781001-19781031',
            'optional': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('optional - testing: finished')
    return log

In [9]:
def test_area():
    log = []
    log.append('\n\n---AREA---')
    cachworkaround = np.random.random_sample()
    for s in [[80,140,-40,359], [49,16,48,17], [49,-180,48,180], [49,0,48,360], [90, -180, -90, 180]]:
        print('area - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'area': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in [[80,140,70], [80,140,70,150,0],[0,0,0,0], [40,20,50,0], [49,-180,48,360]]:
        print('area - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'area': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('area - testing: finished')
    return log

In [10]:
def test_country():
    log = []
    log.append('\n\n---COUNTRY---')
    cachworkaround = np.random.random_sample()
    for s in ['GLOBE', 'ALL', 'NOR', ["NOR","DEU"]]:
        print('country - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'country': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['NORWAY', 'nor',]:
        print('country - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'country': s,
            'date': '19781001-19781031',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('country - testing: finished')
    return log

In [11]:
def test_date(): 
    log = []
    log.append('\n\n---DATE---')
    cachworkaround = np.random.random_sample()
    for s in [None, '20000231', '20000101', '20000101-20000201', ['20000101', '20000103', '20000106'], ['20000102', '20000101']]:
        print('date - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['2000011', '20000100', '00000000', '20000132', '20001301', '20000201-20000101', '20000101/20000201']:
        print('date - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('date - testing: finished')
    return log

In [12]:
def test_period(): 
    log = []
    log.append('\n\n---PERIOD---')
    # not working: ['1979-01-01/1980-08-16']
    cachworkaround = np.random.random_sample()
    for s in [None, '1979-01-01/1980-08-16']:
        print('period - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'period': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['1980-01-02/1980-01-01']:
        print('period - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'period': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('period - testing: finished')
    return log

In [13]:
def test_pressure_level():
    log = []
    log.append('\n\n---PRESSURE LEVEL---')
    cachworkaround = np.random.random_sample()
    for s in [100, '100', [100, 200], ['100', '200']]:
        print('pressure_level - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20000101',
            'pressure_level': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
        
    for s in ['0', '2000']:
        print('pressure_level - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20000101',
            'pressure_level': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
        
    print('pressure_level - testing: finished')
    return log

In [14]:
def test_time():
    log = []
    log.append('\n\n---TIME---')
    cachworkaround = np.random.random_sample()
    for s in [[22,23,0,1,2,3], ["22","23","00","01","02","03"], '', '0', '00', ["00"], [0,12], ["0","12"]]:
        print('time - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20000101-20000201',
            'time': s,
#             'da': False,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['24', '25', '12.5', ]:
        print('time - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20000101-20000201',
            'time': s,
#             'da': False,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('time - testing: finished')
    return log

In [15]:
def test_format():
    log = []
    log.append('\n\n---FORMAT---')
    cachworkaround = np.random.random_sample()
    for s in [None, 'nc']:
        print('format - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20000101-20000201',
            'format': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['csv',]:
        print('format - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20000101-20000201',
            'format': s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(csv_request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in ['netcdf', 'csb', 'txt']:
        print('format - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'date': '20000101-20000201',
            'format': s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('format - testing: finished')
    return log

In [16]:
def test_ymd():
    log = []
    # if one of them is None -> no date selected = the whole dataset - change?
    cachworkaround = np.random.random_sample()
    log.append('\n\n---YEAR---')
    for s in [None, 2000, '2000', ['2000', '2001'], [2000, 2001]]:
        print('y - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': s,
            'month': '01',
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    for s in [0, '20000', '-2000']:
        print('y - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': s,
            'month': '01',
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    log.append('\n\n---MONTH---')
    for s in [None, 1, '1', '01', ['01', '02'], ['1', '2'], [1, 2]]:
        print('m - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': s,
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
    
    for s in ['00', 0, '13']:
        print('m - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': s,
            'day':'01',
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    log.append('\n\n---DAY---')
    for s in ['01', '31', ['01', '02'], ['1', '2'], [1, 2]]:
        print('d - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': '01',
            'day':s,
            'cachworkaround': cachworkaround,
        } 
        try:
            display(request(rd))
            log.append('\n1 request successful: ' + str(s))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
    
    for s in ['00', 0, '32']:
        print('d - testing: ', s)
        rd={
            'variable': 'air_temperature',
            'statid': '11035',
            'year': '2000',
            'month': '01',
            'day':s,
            'cachworkaround': cachworkaround,
        } 
        try: 
            display(request(rd))
            log.append('\n0 request falsely successful: ' + str(s))
        except Exception as e:
            print(e)
            log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('ymd - testing: finished')
    return log

In [17]:
def test_size():
    log = []
    log.append('\n\n---SIZE---')
    cachworkaround = np.random.random_sample()
    s = 'air_temperature 11035'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'statid': '11035',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
    
    s = 'air_temperature 10393'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'statid': '10393',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
        
    s = 'air_temperature 1 month all stations'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'date': '20000101-20000201',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
            
    s = 'air_temperature everything'
    print('size - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'cachworkaround': cachworkaround,
    }
    try: 
        display(request(rd))
        log.append('\n0 request falsely successful: ' + str(s))
    except Exception as e:
        print(e)
        log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
    
    s = 'all variables 1 month all stations'
    print('size - testing: ', s)
    rd={
        'variable': ['air_temperature', 'dew_point_temperature', 'eastward_wind_speed', 'northward_wind_speed', 'relative_humidity', 'specific_humidity', 'wind_from_direction', 'wind_speed'],
        'date': '20000101-20000201',
        'cachworkaround': cachworkaround,
    }
    try: 
        display(request(rd))
        log.append('\n0 request falsely successful: ' + str(s))
    except Exception as e:
        print(e)
        log.append('\n1 request successfully failed: ' + str(s)+ '\n     ' + e.args[0].split('try later b')[-1])
            
    print('size - testing: finished')
    return log

In [25]:
def test_20300():
    log = []
    log.append('\n\n---20300---')
    cachworkaround = np.random.random_sample()
   
    s = 'area'
    print('20300 - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'area': [-77,-42,-78,-41],
#         'date': '20021001-20021101',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
        
    s = '20300*'
    print('20300 - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'statid': '0-20300*',
#         'date': '20021001-20021101',
        'cachworkaround': cachworkaround,
    } 
    try:
        display(request(rd))
        log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
            
    print('20300 - testing: finished')
    return log

In [19]:
def test_optional():
    log = []
    log.append('\n\n---optional---')
    cachworkaround = np.random.random_sample()
    s = ['sonde_type', 'bias_estimate','obs_minus_an','obs_minus_bg',
         'RISE_bias_estimate', 'RICH_bias_estimate', 'RASE_bias_estimate', 'RAOBCORE_bias_estimate', 
         'desroziers_30', 'desroziers_60', 'desroziers_90', 'desroziers_180', 
         'wind_bias_estimate', 'humidity_bias_estimate']
    var = ['temperature', 'temperature', 'temperature', 'temperature',
           'temperature', 'temperature', 'temperature', 'temperature',
           'temperature', 'temperature', 'temperature', 'temperature',
           'u_component_of_wind', 'relative_humidity']
    for i in range(len(s)):
        print('optional - testing: ', s[i])
        rd={
            'variable': var[i],
            'statid': '11035',
            'optional': s[i],
            'date': '20001230-20001231',
            'cachworkaround': cachworkaround,
        } 
        try:
            check_data = request(rd)
            display(check_data)
            if(len(check_data) < 1):
                log.append('\n0 request failed: ' + str(s[i]))
            else:
                log.append('\n1 request successful: ' + str(s[i]))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
            
    print('optional - testing: finished')
    return log

In [20]:
def test_toolbox():
    log = []
    log.append('\n\n---toolbox---')
    cachworkaround = np.random.random_sample()
    s = 'bias_estimate'
    print('toolbox - testing: ', s)
    rd={
        'variable': 'air_temperature',
        'statid': '11035',
        'optional': s,
        'toolbox': 'True',
        'date': '20001230-20001231',
        'cachworkaround': cachworkaround,
    } 
    try:
        check_data = request(rd)
        display(check_data)
        if(len(check_data) < 1):
            log.append('\n0 request failed: ' + str(s))
        else:
            log.append('\n1 request successful: ' + str(s))
    except Exception as e:
        log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
            
    print('toolbox - testing: finished')
    return log

In [21]:
def test_cdm():
    log = []
    log.append('\n\n---cdm---')
    cachworkaround = np.random.random_sample()
    s = ['era5fb/date@hdr', 'sensor_configuration/comments', 'sensor_configuration/observing_method', 'header_table/location_accuracy',
         'crs', 'observations_table/location_method', 'era5fb/obsvalue@body']
    for i in s:
        print('cds - testing: ', i)
        rd={
            'variable': 'temperature',
            'statid': '11035',
            'cdm': [i],
            'date': '20001230-20001231',
            'cachworkaround': cachworkaround,
        } 
        try:
            check_data = cdmrequest(rd)
            display(check_data)
            if(len(check_data['lat'][:]) < 1):
                log.append('\n0 request failed: ' + str(i))
            else:
                log.append('\n1 request successful: ' + str(i))
        except Exception as e:
            log.append('\n0 request failed: ' + str(s) + '\n     ' + e.args[0].split('try later b')[-1])
            
    print('cdm - testing: finished')
    return log

In [22]:
def server_test():
    log = []
    log.append('Testing Server Requests \n')
    log.append('0 = failed | 1 = succeeded \n')
    a = !git rev-parse HEAD
    log.append('GIT revision: ' + str(a) + ' \n')
    log.append(str(date.today())+ ' \n')
    try: log.extend(test_statid())
    except: log.append('\nstatid - test: failed \n')
    try: log.extend(test_variable())
    except: log.append('\nvariable - test: failed \n')
    try: log.extend(test_optional())
    except: log.append('\noptional - test: failed \n')
    try: log.extend(test_area())
    except: log.append('\narea - test: failed \n')
    try: log.extend(test_country())
    except: log.append('\ncountry - test: failed \n')
    try: log.extend(test_date())
    except: log.append('\ndate - test: failed \n')
    try: log.extend(test_period())
    except: log.append('\nperiod - test: failed \n')
    try: log.extend(test_pressure_level())
    except: log.append('\npressure_level - test: failed \n')
    try: log.extend(test_time())
    except: log.append('\ntime - test: failed \n')
    try: log.extend(test_format())
    except: log.append('\nformat - test: failed \n')
    try: log.extend(test_ymd())
    except: log.append('\nymd - test: failed \n')
    log.append('\nsize - test: disabled \n')
#     try: log.extend(test_size())
#     except: log.append('\nsize - test: failed \n')
    try: log.extend(test_20300())
    except: log.append('\n20300 - test: failed \n')
    try: log.extend(test_cdm())
    except: log.append('\ncdm - test: failed \n')
        
    writelog=open(str(date.today()) + '_server_test_log.txt','w')
    writelog.writelines(log)
    writelog.close()

In [23]:
server_test()

2022-10-13 14:44:26,355 INFO Welcome to the CDS
2022-10-13 14:44:26,358 INFO Sending request to https://cds-test.copernicus-climate.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network


statid - testing:  None
{'variable': 'air_temperature', 'statid': None, 'date': '19781001-19781031', 'cachworkaround': 0.3452263419570093}


2022-10-13 14:44:26,440 INFO Request is queued
2022-10-13 14:44:27,470 INFO Request is running
2022-10-13 14:45:41,646 INFO Request is completed
2022-10-13 14:45:41,648 INFO Downloading https://cds-test-download-0000.copernicus-climate.eu/cache-compute-0000/cache/data1/adaptor.comprehensive_upper_air.retrieve-1665665115.8638628-22042-2-239a9051-f4ba-480f-b6a0-505513f6b42e.zip to download.zip (34M)
2022-10-13 14:45:43,002 INFO Download rate 25.1M/s                                                                                            
2022-10-13 14:45:44,421 INFO Reading Profile on slice(None, None, None)


,lat,lon,obs,plev,report_id,station_id,ta,time
0,50.220001,-5.32,0,2970.0,60000022123,CAMBORNE,217.850006,1978-09-30 14:00:00
1,50.220001,-5.32,0,3000.0,60000022123,CAMBORNE,217.649994,1978-09-30 14:00:00
2,50.220001,-5.32,0,3980.0,60000022123,CAMBORNE,215.050003,1978-09-30 14:00:00
3,50.220001,-5.32,0,5000.0,60000022123,CAMBORNE,215.250000,1978-09-30 14:00:00
4,50.220001,-5.32,0,6550.0,60000022123,CAMBORNE,215.449997,1978-09-30 14:00:00
...,...,...,...,...,...,...,...,...
1712,50.220001,-5.32,0,81100.0,20000027688,CAMBORNE,277.200012,1978-10-31 12:00:00
1713,50.220001,-5.32,0,85000.0,20000027688,CAMBORNE,277.200012,1978-10-31 12:00:00
1714,50.220001,-5.32,0,86600.0,20000027688,CAMBORNE,277.799988,1978-10-31 12:00:00
1715,50.220001,-5.32,0,100000.0,20000027688,CAMBORNE,286.200012,1978-10-31 12:00:00


2022-10-13 14:45:44,558 INFO Welcome to the CDS
2022-10-13 14:45:44,559 INFO Sending request to https://cds-test.copernicus-climate.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2022-10-13 14:45:44,622 INFO Request is queued


...

2022-10-13 15:03:41,337 INFO Welcome to the CDS
2022-10-13 15:03:41,339 INFO Sending request to https://cds-test.copernicus-climate.eu/api/v2/resources/insitu-comprehensive-upper-air-observation-network
2022-10-13 15:03:41,401 INFO Request is queued


cds - testing:  era5fb/obsvalue@body


2022-10-13 15:03:42,435 INFO Request is completed
2022-10-13 15:03:42,436 INFO Downloading https://cds-test-download-0000.copernicus-climate.eu/cache-compute-0000/cache/data1/adaptor.comprehensive_upper_air.retrieve-1665666221.792629-30539-6-12a22371-f4e7-4ffc-93a9-e53c86328223.zip to download.zip (30.5K)
2022-10-13 15:03:42,488 INFO Download rate 593.5K/s                                                                                           


File: <HDF5 file "dest_0-20001-0-11035_air_temperature.nc" (mode r)>
Filesize: 0.03 MB
Filename: ./test1/dest_0-20001-0-11035_air_temperature.nc
(G)roups/(V)ariables: 

 - V | lat__________________________________________ : : (252,)
 - V | lon__________________________________________ : : (252,)
 - V | obs__________________________________________ : : (252,)
 - V | plev_________________________________________ : : (252,)
 - V | report_id____________________________________ : : (252, 11)
 - V | station_id___________________________________ : : (252, 25)
 - V | string11_____________________________________ : : (11,)
 - V | string25_____________________________________ : : (25,)
 - V | ta___________________________________________ : : (252,)
 - V | time_________________________________________ : : (252,)

cdm - testing: finished
